In [ ]:
#import libraries
import numpy as np
import pandas as pd

distance_matrix = pd.read_csv("data/final_distance.csv")
df=pd.DataFrame(distance_matrix)
dist_matrix=df.to_numpy()

def calculate_distance_matrix(df,points): #distance_matrix,수거지개수
    """
    Calculate the distance matrix between coordinates.
    """
    num_points = points
    dist_matrix =df.to_numpy()
    return dist_matrix

def calculate_total_distance(route, dist_matrix): #특정 경로= route
    """
    Calculate the total distance of a given route using the distance matrix.
    """
    total_distance = 0
    num_points = len(route)

    for i in range(num_points - 1):
        current_node = route[i]
        next_node = route[i + 1]
        total_distance += dist_matrix[current_node, next_node]

    return total_distance


def nearest_neighbor(dist_matrix, demands, capacity,num_car=5,last_node=0):
    """
    Apply the Nearest Neighbor heuristic to find initial routes for VRP.
    1.현재 가지고 있는 차량들 개수로 start_node=0 (차고지)로 설정
    2.한바퀴돌고 나서 다음부터는 매립지를 기준으로 start_node=차고지로 설정
    """
    num_points = dist_matrix.shape[0]
    visited = np.zeros(num_points-2, dtype=bool) #모두 false로 저장
    visited=np.hstack([np.ones(1,dtype=bool),visited,np.ones(1,dtype=bool)]) # 매립지는 True로 저장하기
    routes = []
    capa=[]
    num_car=5
    while (np.sum(visited) < num_points):
        if len(routes)>=num_car:
            last_node=len(df)-1 #차고지로 설정
        current_node = last_node  # Start at node 0
        current_capacity = 0
        route = [current_node]
        visited[current_node] = True #첫번째 node0 방문

        while current_capacity + demands[current_node] <= capacity:
            current = route[-1] #현재노드
            nearest = None
            min_dist = float('inf') #무한값

            for neighbor in np.where(~visited)[0]:#visit false인곳 반복
                #capa 가능한 & 최소 거리인 곳저장과정
                if demands[neighbor] + current_capacity <= capacity and dist_matrix[current, neighbor] < min_dist: 
                    nearest = neighbor #가까운 노드 저장
                    min_dist = dist_matrix[current, neighbor] #가장 가까운 거리 저장
            #남은 곳 어느곳도 capa를 담지못해서 break
            if nearest is None:
                break
            #route에 nearest 저장, visited ture변환, capa 저장
            route.append(nearest)
            visited[nearest] = True
            current_capacity += demands[nearest]
        #모든 경로들 저장
        route.append(len(df)-1)
        routes.append(route)
        capa.append(current_capacity)

    return routes,capa


def format_output(routes):
    """
    화살표로 노드 구현
    """
    num=0
    for i in range(len(routes)):
        dis=calculate_total_distance(routes[i], dist_matrix)
        num+=dis 
        print(f"Route{i}:{dis}")
        print(*routes[i],sep="->")
        print("")
    print("Objective:"+str(num))

        
#    return routes

def vrp_solver(distance_matrix,capacity):
    """
    Solve the VRP using the provided filename for coordinates and vehicle capacity.
    """
    #distance_matrix = pd.read_csv(filename)
    display(distance_matrix)
    dist_matrix=df.to_numpy()
    demands=np.ones(len(df))
    routes,capa = nearest_neighbor(dist_matrix, demands, capacity,2)
    formatted_routes = format_output(routes)
    return routes,capa


In [ ]:
routes,capa=vrp_solver(df,15)